# CS-433 Project 1 by group twohundredbpm

## Useful imports

In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from implementations import *
from cost import *
from gradients import *
from preprocessing import *
from cross_validation import *
%load_ext autoreload
%autoreload 2

## Getting data

In [ ]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'Data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [ ]:
DATA_TEST_PATH = 'Data/test.csv'  # TODO: download train data and supply path here
y_test, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

## Running cross-validation to get the best hyper-parameters

In [ ]:
tX_cross = tX.copy()
tX_cross = preprocess(tX_cross)

In [ ]:
print(np.logspace(-5, -2, 4))

In [ ]:
for idx in range(len(tX)):
    train_x_jet = tX_cross[idx]
    train_y_jet = y[idx]
    test_x_jet = tX_test[idx]
    
    _, best_params = cross_validation(train_y_jet, train_x_jet,
                     degrees=np.arange(10,14), lambdas=np.logspace(-5, -2, 4))
    
    print("for model {}, the best parameters are : \n",
          "degree = ", best_params[0], " lambda = ", best_params[1])

## Building the model and retrieving prediction

In [ ]:
# setting up parameters
degrees = [11, 12, 13, 12]
lambdas = [0.001009312, 0.001009312, 1.1212e-05, 0.0000696969]

# Removing irrelevant columns from the jet groups:
tX = preprocess(tX)
tX_test = preprocess(tX_test)

# Instancing predictions
y_pred = []

In [ ]:
# Building models
for idx in range(len(tX)):
    print("Beginning training on model ", idx+1)

    # extracting the values of specific group :
    train_x_jet = tX[idx]
    train_y_jet = y[idx]
    test_x_jet = tX_test[idx]

    # Polynomial feature expansion :
    tX_train_poly = polynomial_expansion(train_x_jet, degrees[idx])
    tX_test_poly = polynomial_expansion(test_x_jet, degrees[idx])

    w_, loss_ = ridge_regression(train_y_jet, tX_train_poly, lambdas[idx])

    accuracy_ = compute_accuracy(train_y_jet, tX_train_poly, w_)
    print('The accuracy of model {} is equal to {}'.format(int(idx+1), accuracy_))

    y_pred_jet = predict_labels(w_, tX_test_poly)

    y_pred.append(y_pred_jet)

In [ ]:
%%capture
# Building back the prediction array from the sorted ones
y_total = np.zeros(len(np.hstack(ids_test)))

min_id_test = min(np.hstack(ids_test))

ids_total = np.arange(len(y_total))
ids_total += min_id_test

for jet_num in range(len(y)):
    for j in range(len(y_pred[jet_num])):
        y_total[ids_test[jet_num][j] - min_id_test] = y_pred[jet_num][j]

y_total.reshape(-1, 1)

## Building output

In [ ]:
OUTPUT_PATH = "Data/doto.csv"
print('Building output file in ', OUTPUT_PATH)
create_csv_submission(ids_total, y_total, OUTPUT_PATH)